In [3]:
"""
주말 과제 : 3장에 있는 퍼셉트론 코드 구현를 구현하여 mnist로 손글씨 인식 되는지 코드를 짜기. 
* 추가 : 활성화 함수(시그모이드, ReLU)를 바꿔서 성능 비교하기.
조건 : numpy만 이용
제출 파일 형식 : ipynb 또는 py 파일과 실행 화면. 왜 둘중에 ipynb과 py파일을 썻는지도 서술하기
ps : 2장에 있는 연습 코드들은 파이썬 기초 다루시는 분들은 꼭 해보시길 바랍니다.
물론 인터넷에 코드가 다있습니다 하지만 보지않고 본인 스스로 하시길 바랍니다. 
어떤 방법을 써도 본인 마음입니다. 하지만 사용에 대한 이유가 있어야합니다~ 
예를 들어 윈도우에서 한 이유, 서버에서 한 이유 등등...입니다!
에러뜨면 무작정 질문하지 마시고 한번 서칭후에 질문 부탁드립니다!


코드 짠 사람 : 김광재

핵심은 이전 mnist에 사용하던 가중치, 편향, 입력 데이터를 넘파이 저장 함수로 저장해 피클 모듈을 안써도 되게 만드는 것,
그리고 시그모이드와 소프트 맥스 모듈에서 가져온 정의식도 코드 내부로 옮기는 것이다.
~.ipynb 형식을 썼냐면, 코드 셀이 서로 독립적이라 다른 파이썬 파일을 만들지 않고 코드 셀에 코드를 집어넣어 백업을 만들 수 있고, 터미널을 사용하지 않고 바로 결과가 나오기 때문이다.
서버가 아닌 개인 컴퓨터 내에서 작성했다. 네트워크 연결이 안될때도 코드를 작성하기 위함이고 코드도 네트워크에 영향을 받지 않는 코드로 만들기 위해서다.
 
"""

# coding: utf-8

import numpy as np

'''
import sys, os는 부모 디렉터리의 파일을 가져올 수 있도록 설정하는 것과, 가중치값 저장된 피클 파일, 가져올 때 씀. -> 폴더 하나에 숙제랑 데이터 파일 다 박으면 안서도 되겠다.
pickle은 피클 파일 가져올 때 씀.
load_mnist는 mnist 데이터 셋 가져올 때 씀.
소프트맥스랑 시그모이드는 거기있는 정의 가져올 때 씀. 이건 정의문을 아예 가져오는 형식으로 진행하자. -> 가져왔음
'''

# from common.functions import sigmoid, softmax 에서 가져온 시그모이드랑 소프트맥스 정의식.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

#여기 정규화(0~255값을 0~1로 변화)를 했는데 이건 입력 데이터 변환하는 전처리의 작업중 하나임.
# 흐음, mnist.pkl 파일에 이미지 파일들이 저장되어 있는 것 같다. 이걸 변환시켜야 한다. 아니면 피클파일을 어찌어찌 직접 다시 만들던가, 아니면 피클의 원본 파일을 얻어야되.
# .gz 파일들은 뭐지? 어떻게 못보나? 그냥 numpy 배열을 저장하는거 찾았다!
# load_mnist 함수 보니까 넘파이 배열로 피클 파일 만들어서 서장하더라. 그게 mnist.pkl. 거기서 값 가져와 쓴다네. 그러면, load_mnist 값을 출력시켜서 넘파이 배열 파일로 저장하면 꺼내쓸수있겠다.

'''
그래서 예제 수행에 필요한 데이터셋 파일을 /dataset/ 디렉터리에 올려뒀습니다.
혹 사이트가 다운되어 데이터를 받을 수 없다면 아래 파일 4개를 각자의 <예제 소스 홈>/dataset/ 디렉터리 밑에 복사해두면 됩니다. ^__^
-> 라벨 데이터는 다 있다. 이걸 전부 network 변수에 저장하면 된다!. 그러기 위해선 gz파일들을 전부 network 변수에 저장하는 방법을 알면 된다.
'''

'''
가중치 피클 파일이 만들어져있다. 피클 파일들의 원본은 뭔지 알아봐야겠다. 아니면 피클 파일을 전부 출력해서 봐달라고 하면 되지않나? 리스트 형식으로.
일단 네트워크 변수의 형식을 알아봐야겠네. numpy 배열같다. 이러면 예기가 쉽다. 넘파이에 세이브 파일 만드는거 있네?
넘파이 파일을 어떻게 네트워크에 넘기는지도 알아봐야겠다.
정의문만 남겨뒀지 코드에는 쓰이지 않음.
'''

# 이전 예제에서 가져온 추론 함수 정의식
def predict(network, x):

    #network = list(map(float, network))    #network가 잘 들어가는지 확인용 함수
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y


'''
# 이전에 쓰던 리스트 파일 저장함수. 
# x, t 리스트 일일이 저장해서 get_data() 안쓸려는 구성.
# 타입 오류 때문에 폐기하고 np.savez 사용함.
# open file in write mode
with open('D:\파이썬연습\deeplearn\mnist_homework\mnist_x', 'w') as fp:
    for item in x:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('x Done')

with open('D:\파이썬연습\deeplearn\mnist_homework\mnist_t', 'w') as fp:
    for item in t:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('t Done')
'''




"""
# print(network)    #network 변수 출력값을 다 알아보기 위함.
np.savez('D:\파이썬연습\deeplearn\mnist_homework', W1=network['W1'], W2=network['W2'],W3=network['W3'],b1=network['b1'],b2=network['b2'],b3=network['b3'])
# 넘파이 배열 저장 함수 적용. 터미널 자기 위치를 잘 봐야됨. 안그럼 넘파이 배열 저장된 함수 .npy가 다른데 생김
# np.savez로 직접 네트워크 데이터를 옮겨서 npz파일로 저장해 데이터를 담음.
#출처: https://rfriend.tistory.com/358 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]
np.save('D:\파이썬연습\deeplearn\mnist_x',x)
np.save('D:\파이썬연습\deeplearn\mnist_t',t)

networkl = np.load('D:\파이썬연습\deeplearn\mnist_homework.npz')
x = np.load('D:\파이썬연습\deeplearn\mnist_x.npy')
t = np.load('D:\파이썬연습\deeplearn\mnist_x.npy')

"""

# 이전에 잘되던 network, x, t 값들 넘파이 파일로 저장해둔 것들을 다시 불러옴.
# 반드시 파이썬 코드와 같은 폴더 내에 위치해야됨. 안그럼 경로 오류로 파일을 못찾는 경우가 많음.
network = np.load('mnist_homework.npz')
x = np.load('mnist_x.npy')
t = np.load('mnist_t.npy')

#print(x[1]) # x가 잘 들어갔는지 하나만 확인함.
#x = x.astype(np.unit58)    # 이전 데이터 타입 오류 고칠려던 코드. 실패함.
#print(t)
#x = np.array(x)

accuracy_cnt = 0    # 초기값 설정
for i in range(len(x)):#훈련이미지 훈련레이블 중에서 훈련 이미지를 가져옴. t는 훈련레이블 0~6만?
    #print("x[i] = ",x[i]) # 오류확인용
    y = predict(network, x[i])  # 0~9 의 가능성 확률을 계산해서 넘김
    #print(np.argmax(y))    # 오류확인용 코드. 최대값을 잘 넘기나 확인
    p= np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다. 레이블이 정답이란 의미임. 이거 정답을 주는 처리네?
    if p == t[i]:   # 정답이라면 정확도를 1 높인다.
        accuracy_cnt += 1
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352
